<p align="center">
    <img src="https://soudevcon.com.br/wp-content/uploads/2025/01/logo-soudevcon-1000-branco.png" width="300px"/><br><br>
</p>

## SouDevCon - 2025
### A Celebração das Linguagens de Programação!

# Aprendizado de Máquina com Dados em Larga Escala.

__Conteúdo:__

- Parte I: Spark DataFrame API
- Parte II: Spark SQL

In [1]:
from IPython.core.display import HTML
display(HTML('<style>pre {white-space: pre !important;}</style>'))

## Parte I: Spark DataFrame API

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark = (
    SparkSession
    .builder
    .master('spark://spark-master:7077')
    .appName('SparkDataFrameAPI')
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/07 13:29:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
enem_df = (
    spark
    .read
    .csv(
        'RESULTADOS_2024.csv',
        sep=';',
        header=True,
        inferSchema=True,
        encoding='latin1'
    )
)

enem_df.show(5, truncate=False)

25/08/07 13:29:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+------+---------+----------------+-----------------------+---------+---------+----------------------+------------------+---------------+------------------+----------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------------------------------------+---------------------------------------------+---------------------------------------------+---------------------------------------------+---------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+
|NU_SEQUENCIAL|NU_ANO|CO_ESCOLA|CO_MUNICIPIO_ESC|NO_MUNICIPIO_ESC       |CO_UF_ESC|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|TP_SIT_F

In [5]:
enem_df.count()

4332944

In [6]:
enem_df.printSchema()

root
 |-- NU_SEQUENCIAL: integer (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer (nullable = true)
 |-- SG_UF_PROVA: string (nullable = true)
 |-- TP_PRESENCA_CN: integer (nullable = true)
 |-- TP_PRESENCA_CH: integer (nullable = true)
 |-- TP_PRESENCA_LC: integer (nullable = true)
 |-- TP_PRESENCA_MT: integer (nullable = true)
 |-- CO_PROVA_CN: integer (nullable = true)
 |-- CO_PROVA_CH: integer (nullable = true)
 |-- CO_PROVA_LC: integer (nullable = true)
 |-- CO_PROVA_MT: integer 

In [7]:
# Visualizando a quantidade de dados por estado de realizacao da prova (SG_UF_PROVA)
enem_df.groupBy(
    'SG_UF_PROVA'
).count().orderBy('count', ascending=False).show(5)

[Stage 6:===========================================>             (10 + 3) / 13]

+-----------+------+
|SG_UF_PROVA| count|
+-----------+------+
|         SP|647215|
|         MG|393824|
|         BA|376845|
|         RJ|289397|
|         RS|279039|
+-----------+------+
only showing top 5 rows



In [8]:
# Visualizando a media
# das notas de LC, MT e REDACAO por estado de realizacao das provas
%time
enem_df.groupBy(
    'SG_UF_PROVA'
).agg(
    F.mean(F.col('NU_NOTA_LC')).alias('MEDIA_NOTA_LC'),
    F.mean(F.col('NU_NOTA_MT')).alias('MEDIA_NOTA_MT'),
    F.mean(F.col('NU_NOTA_REDACAO')).alias('MEDIA_NOTA_REDACAO')
).orderBy('SG_UF_PROVA').show(5)

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 4.77 μs


[Stage 9:===========================================>             (10 + 3) / 13]

+-----------+------------------+------------------+------------------+
|SG_UF_PROVA|     MEDIA_NOTA_LC|     MEDIA_NOTA_MT|MEDIA_NOTA_REDACAO|
+-----------+------------------+------------------+------------------+
|         AC|506.69141881471694| 489.9609728707371| 589.4910773298084|
|         AL|503.99079384934555| 505.4732616476004| 591.9077401836654|
|         AM| 499.1964784757736|482.56220598650646| 540.9731652670646|
|         AP|502.99491761429573| 479.6627616105242| 579.2629380366675|
|         BA| 508.7301438327707|  499.372481400336| 598.7217970453785|
+-----------+------------------+------------------+------------------+
only showing top 5 rows



In [9]:
# Visualizando desvio padrao
# das notas de LC, MT e REDACAO por estado de realizacao das provas
%time
enem_df.groupBy(
    'SG_UF_PROVA'
).agg(
    F.std(F.col('NU_NOTA_LC')).alias('STD_NOTA_LC'),
    F.std(F.col('NU_NOTA_MT')).alias('STD_NOTA_MT'),
    F.std(F.col('NU_NOTA_REDACAO')).alias('STD_NOTA_REDACAO')
).orderBy('SG_UF_PROVA').show(5)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 4.05 μs


[Stage 12:===================================================>    (12 + 1) / 13]

+-----------+-----------------+------------------+------------------+
|SG_UF_PROVA|      STD_NOTA_LC|       STD_NOTA_MT|  STD_NOTA_REDACAO|
+-----------+-----------------+------------------+------------------+
|         AC|70.06169041603373| 95.22926904934953|212.25267461771952|
|         AL|72.74449160311147|106.89082858766297|  242.096241563416|
|         AM|72.64815076108428| 93.91156444915472|224.56191442909383|
|         AP|71.01471708969164| 91.05994355623126| 227.0072301915263|
|         BA|70.50809820417075|101.99542187756019|223.08724325934202|
+-----------+-----------------+------------------+------------------+
only showing top 5 rows



In [10]:
%time
enem_df.groupBy(
    'SG_UF_PROVA'
).agg(
    F.mean(F.col('NU_NOTA_REDACAO')).alias('MEDIA_NOTA_REDACAO'),
    F.min(F.col('NU_NOTA_REDACAO')).alias('MENOR_NOTA_REDACAO'),
    F.max(F.col('NU_NOTA_REDACAO')).alias('MAIOR_NOTA_REDACAO')
).orderBy('SG_UF_PROVA').show(5)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 3.34 μs


[Stage 15:===========================================>            (10 + 3) / 13]

+-----------+------------------+------------------+------------------+
|SG_UF_PROVA|MEDIA_NOTA_REDACAO|MENOR_NOTA_REDACAO|MAIOR_NOTA_REDACAO|
+-----------+------------------+------------------+------------------+
|         AC| 589.4910773298084|                 0|               980|
|         AL| 591.9077401836654|                 0|              1000|
|         AM| 540.9731652670646|                 0|               980|
|         AP| 579.2629380366675|                 0|               980|
|         BA| 598.7217970453785|                 0|               980|
+-----------+------------------+------------------+------------------+
only showing top 5 rows



In [11]:
# Visualizando a media
# das notas de LC, MT e REDACAO de realizacao das provas da regiao sudeste
%time
enem_df.where(
    F.col('SG_UF_PROVA').isin(['SP', 'MG', 'RJ', 'ES'])
).groupBy(
    'SG_UF_PROVA'
).agg(
    F.mean(F.col('NU_NOTA_LC')).alias('MEDIA_NOTA_LC'),
    F.mean(F.col('NU_NOTA_MT')).alias('MEDIA_NOTA_MT'),
    F.mean(F.col('NU_NOTA_REDACAO')).alias('MEDIA_NOTA_REDACAO')
).orderBy('SG_UF_PROVA').show(5)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.34 μs


[Stage 18:===========================================>            (10 + 3) / 13]

+-----------+-----------------+-----------------+------------------+
|SG_UF_PROVA|    MEDIA_NOTA_LC|    MEDIA_NOTA_MT|MEDIA_NOTA_REDACAO|
+-----------+-----------------+-----------------+------------------+
|         ES|535.4067699940247| 546.226067924818| 650.5578590957649|
|         MG|538.0591089142654|551.9026473637185| 664.7805011078916|
|         RJ|538.4875202137388|543.0368571936972| 641.5937565914364|
|         SP|547.6865680589616|559.7713125587762| 652.2811404065163|
+-----------+-----------------+-----------------+------------------+



In [12]:
# MEDIA DAS NOTAS POR PROVA - SOMENTE CANDIDATOS QUE ZERARAM A REDAÇÃO da Região Sudeste
%time
enem_df.where(
    (F.col('SG_UF_PROVA').isin(['SP', 'MG', 'RJ', 'ES'])) &
    (F.col('NU_NOTA_REDACAO') == 0)
).groupBy(
    'SG_UF_PROVA'
).agg(
    F.mean(F.col('NU_NOTA_LC')).alias('MEDIA_NOTA_LC'),
    F.mean(F.col('NU_NOTA_MT')).alias('MEDIA_NOTA_MT'),
).orderBy('SG_UF_PROVA').show(5)

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 2.86 μs


[Stage 21:===============================================>        (11 + 2) / 13]

+-----------+------------------+------------------+
|SG_UF_PROVA|     MEDIA_NOTA_LC|     MEDIA_NOTA_MT|
+-----------+------------------+------------------+
|         ES|478.50738825267814| 473.0251497005988|
|         MG| 466.7090728914335|464.30234123500145|
|         RJ|470.29895035163185| 458.5498177740398|
|         SP|480.95447657841135|474.65758952149565|
+-----------+------------------+------------------+



In [13]:
# Dados dos participantes do ENEM 2024
participantes_df = (
    spark
    .read
    .csv(
        'PARTICIPANTES_2024.csv',
        sep=';',
        header=True,
        inferSchema=True,
        encoding='latin1'
    )
)
participantes_df.show(5,truncate=False)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------------+------------------+-----------+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ENSINO|IN_TREINEIRO|CO_MUNICIPIO_PROVA|NO_MUNICIPIO_PROVA|CO_UF_PROVA|SG_UF_PROVA|Q001|Q002|Q003|Q004|Q005|Q006|Q007|Q008|Q009|Q010|Q011|Q012|Q013|Q014|Q015|Q016|Q017|Q018|Q019|Q020|Q021|Q022|Q023|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------------+------------------+-----------+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|210062064233|2024  |5         

In [14]:
# Visualizando participantes por faixa etaria
participantes_df.count()

4332944

In [15]:
#visualizando participantes por sexo
participantes_df.groupBy(
    'TP_SEXO'
).count().show()

[Stage 30:====>                                                   (1 + 11) / 12]

+-------+-------+
|TP_SEXO|  count|
+-------+-------+
|      F|2624621|
|      M|1708323|
+-------+-------+



In [16]:
#Visualizando inscritos por raça
participantes_df.groupBy(
    'TP_COR_RACA'
).count().orderBy('TP_COR_RACA').show()

[Stage 33:==============>                                          (3 + 9) / 12]

+-----------+-------+
|TP_COR_RACA|  count|
+-----------+-------+
|          0|  50632|
|          1|1791884|
|          2| 534653|
|          3|1863437|
|          4|  62419|
|          5|  29919|
+-----------+-------+



In [17]:
# Número de Candidatos inscritos por Sexo e RACA
participantes_df.groupBy(
    'TP_SEXO', 'TP_COR_RACA'
).count().orderBy('TP_COR_RACA').show()

[Stage 36:==============================================>         (10 + 2) / 12]

+-------+-----------+-------+
|TP_SEXO|TP_COR_RACA|  count|
+-------+-----------+-------+
|      F|          0|  29306|
|      M|          0|  21326|
|      M|          1| 700104|
|      F|          1|1091780|
|      F|          2| 311015|
|      M|          2| 223638|
|      F|          3|1135887|
|      M|          3| 727550|
|      F|          4|  39664|
|      M|          4|  22755|
|      M|          5|  12950|
|      F|          5|  16969|
+-------+-----------+-------+



In [18]:
## JOIN entre os dfs enem e participantes que fizeram a prova
## no municipio de Pedreiras-MA (CO_MUNICIPIO_PROVA==2108207)
## com NU_NOTA_REDACAO > 0

In [19]:
participantes = participantes_df.filter(
    (F.col('CO_MUNICIPIO_PROVA') == 2108207)
).select(
    F.col('CO_MUNICIPIO_PROVA'),
    F.col('TP_SEXO'),
    F.col('TP_COR_RACA'),
)
participantes.count()

2282

In [20]:
enem = enem_df.filter(
    (F.col('CO_MUNICIPIO_PROVA') == 2108207) &
    (F.col('NU_NOTA_CN').isNotNull()) &
    (F.col('NU_NOTA_CH').isNotNull()) &
    (F.col('NU_NOTA_LC').isNotNull()) &
    (F.col('NU_NOTA_MT').isNotNull()) &
    (F.col('NU_NOTA_REDACAO') > 0)
).select(
    F.col('CO_MUNICIPIO_PROVA'),
    F.col('NU_NOTA_CN'),
    F.col('NU_NOTA_CH'),
    F.col('NU_NOTA_LC'),
    F.col('NU_NOTA_MT'),
    F.col('NU_NOTA_REDACAO')
)
enem.count()

1450

In [21]:
join_df = participantes.join(
    enem,
    on=['CO_MUNICIPIO_PROVA'],
    how='inner'
)
join_df.limit(5).show(truncate=False)

[Stage 46:===========================================>            (10 + 3) / 13]

+------------------+-------+-----------+----------+----------+----------+----------+---------------+
|CO_MUNICIPIO_PROVA|TP_SEXO|TP_COR_RACA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------------+-------+-----------+----------+----------+----------+----------+---------------+
|2108207           |F      |3          |441.6     |529.9     |532.8     |505.6     |640            |
|2108207           |F      |3          |409.5     |411.9     |498.0     |437.9     |640            |
|2108207           |F      |3          |581.1     |520.2     |555.8     |645.7     |860            |
|2108207           |F      |3          |590.5     |523.8     |533.6     |617.6     |680            |
|2108207           |F      |3          |485.5     |344.8     |343.9     |408.5     |460            |
+------------------+-------+-----------+----------+----------+----------+----------+---------------+



In [23]:
# Media nota matematica por sexo
join_df.groupBy(
    'TP_SEXO'
).agg(
    F.mean(F.col('NU_NOTA_LC')).alias('MEDIA_NOTA_LC'),
    F.mean(F.col('NU_NOTA_MT')).alias('MEDIA_NOTA_MT')
).show()

[Stage 51:===========================================>            (10 + 3) / 13]

+-------+------------------+------------------+
|TP_SEXO|     MEDIA_NOTA_LC|     MEDIA_NOTA_MT|
+-------+------------------+------------------+
|      F|504.24737931081427|502.93799999990705|
|      M|504.24737931051396|502.93799999985504|
+-------+------------------+------------------+



In [24]:
# nota LC pela raça
join_df.groupBy(
    'TP_COR_RACA'
).agg(
    F.mean(F.col('NU_NOTA_LC')).alias('MEDIA_NOTA_LC')
).show()

[Stage 60:===========================================>            (10 + 3) / 13]

+-----------+------------------+
|TP_COR_RACA|     MEDIA_NOTA_LC|
+-----------+------------------+
|          1|504.24737931025453|
|          3| 504.2473793108308|
|          5|504.24737931034633|
|          4| 504.2473793103383|
|          2| 504.2473793102517|
|          0| 504.2473793103409|
+-----------+------------------+



In [25]:
spark.stop()

## Parte II: Spark SQL

In [26]:
spark = ( 
    SparkSession
    .builder
    .master("spark://spark-master:7077")
    .appName('spark_sql')
    .getOrCreate()
)

In [27]:
enem_df = (
    spark
    .read
    .option('header', True)
    .option('inferSchema', True)
    .option('sep', ';')
    .option('encoding', 'latin1')
    .csv('RESULTADOS_2024.csv')
)

enem_df.createOrReplaceTempView('enem_tb')

In [28]:
# Visualizando a quantidade de dados por estado de realizacao da prova (SG_UF_PROVA)
spark.sql(
    '''
        SELECT
            SG_UF_PROVA,
            COUNT(SG_UF_PROVA) AS COUNT_UF
        FROM enem_tb
        GROUP BY SG_UF_PROVA
        ORDER BY COUNT_UF
    '''
).show(5, truncate=False)

[Stage 2:===========================================>             (10 + 3) / 13]

+-----------+--------+
|SG_UF_PROVA|COUNT_UF|
+-----------+--------+
|RR         |12695   |
|AC         |26424   |
|AP         |30650   |
|TO         |34539   |
|RO         |38609   |
+-----------+--------+
only showing top 5 rows



In [29]:
# Visualizando a media
# das notas de LC, MT e REDACAO por estado de realizacao das provas
spark.sql(
    '''
        SELECT
            SG_UF_PROVA,
            MEAN(NU_NOTA_LC) AS MEDIA_NOTA_LC,
            MEAN(NU_NOTA_MT) AS MEDIA_NOTA_MT,
            MEAN(NU_NOTA_REDACAO) AS MEDIA_NOTA_REDACAO
        FROM enem_tb
        GROUP BY SG_UF_PROVA
        ORDER BY SG_UF_PROVA
    '''
).show(5, truncate=False)

[Stage 5:===========================================>             (10 + 3) / 13]

+-----------+------------------+------------------+------------------+
|SG_UF_PROVA|MEDIA_NOTA_LC     |MEDIA_NOTA_MT     |MEDIA_NOTA_REDACAO|
+-----------+------------------+------------------+------------------+
|AC         |506.69141881471694|489.9609728707371 |589.4910773298084 |
|AL         |503.9907938493455 |505.4732616476005 |591.9077401836654 |
|AM         |499.1964784757735 |482.56220598650646|540.9731652670646 |
|AP         |502.99491761429573|479.6627616105242 |579.2629380366675 |
|BA         |508.7301438327707 |499.372481400336  |598.7217970453785 |
+-----------+------------------+------------------+------------------+
only showing top 5 rows



In [30]:
# Visualizando desvio padrao
# das notas de LC, MT e REDACAO por estado de realizacao das provas
spark.sql(
    '''
        SELECT
            SG_UF_PROVA,
            STD(NU_NOTA_LC) AS STD_NOTA_LC,
            STD(NU_NOTA_MT) AS STD_NOTA_MT,
            STD(NU_NOTA_REDACAO) AS STD_NOTA_REDACAO
        FROM enem_tb
        GROUP BY SG_UF_PROVA
        ORDER BY SG_UF_PROVA
    '''
).show(5, truncate=False)

[Stage 8:===========================================>             (10 + 3) / 13]

+-----------+-----------------+------------------+------------------+
|SG_UF_PROVA|STD_NOTA_LC      |STD_NOTA_MT       |STD_NOTA_REDACAO  |
+-----------+-----------------+------------------+------------------+
|AC         |70.06169041603373|95.22926904934953 |212.2526746177195 |
|AL         |72.74449160311147|106.89082858766298|242.096241563416  |
|AM         |72.64815076108428|93.91156444915472 |224.5619144290938 |
|AP         |71.01471708969164|91.05994355623126 |227.00723019152628|
|BA         |70.50809820417075|101.99542187756019|223.087243259342  |
+-----------+-----------------+------------------+------------------+
only showing top 5 rows



In [31]:
# Visualizando a media
# das notas de LC, MT e REDACAO de realizacao das provas da regiao sudeste
spark.sql(
    '''
        SELECT
            SG_UF_PROVA,
            MEAN(NU_NOTA_LC) AS MEDIA_NOTA_LC,
            MEAN(NU_NOTA_MT) AS MEDIA_NOTA_MT,
            MEAN(NU_NOTA_REDACAO) AS MEDIA_NOTA_REDACAO
        FROM enem_tb
        WHERE SG_UF_PROVA IN ('SP', 'MG', 'ES', 'RJ')
        GROUP BY SG_UF_PROVA
    '''
).show(5, truncate=False)

[Stage 11:===============================================>        (11 + 2) / 13]

+-----------+-----------------+-----------------+------------------+
|SG_UF_PROVA|MEDIA_NOTA_LC    |MEDIA_NOTA_MT    |MEDIA_NOTA_REDACAO|
+-----------+-----------------+-----------------+------------------+
|SP         |547.6865680589616|559.7713125587762|652.2811404065163 |
|ES         |535.4067699940247|546.2260679248182|650.5578590957649 |
|MG         |538.0591089142655|551.9026473637185|664.7805011078916 |
|RJ         |538.4875202137385|543.0368571936972|641.5937565914364 |
+-----------+-----------------+-----------------+------------------+



In [32]:
# MEDIA DAS NOTAS POR PROVA - SOMENTE CANDIDATOS QUE ZERARAM A REDAÇÃO da Região Sudeste
spark.sql(
    '''
        SELECT
            SG_UF_PROVA,
            MEAN(NU_NOTA_LC) AS MEDIA_NOTA_LC,
            MEAN(NU_NOTA_MT) AS MEDIA_NOTA_MT
        FROM enem_tb
        WHERE 
            SG_UF_PROVA IN ('SP', 'MG', 'ES', 'RJ') AND
            NU_NOTA_REDACAO = 0
        GROUP BY SG_UF_PROVA 
    '''
).show(5, truncate=False)

[Stage 14:===========================================>            (10 + 3) / 13]

+-----------+------------------+------------------+
|SG_UF_PROVA|MEDIA_NOTA_LC     |MEDIA_NOTA_MT     |
+-----------+------------------+------------------+
|SP         |480.95447657841135|474.65758952149565|
|ES         |478.50738825267825|473.0251497005988 |
|MG         |466.7090728914336 |464.3023412350015 |
|RJ         |470.29895035163185|458.5498177740398 |
+-----------+------------------+------------------+



In [33]:
# Dados dos participantes do ENEM 2024
participantes_df = (
    spark
    .read
    .csv(
        'PARTICIPANTES_2024.csv',
        sep=';',
        header=True,
        inferSchema=True,
        encoding='latin1'
    )
)
participantes_df.createOrReplaceTempView('participantes_tb')

In [34]:
#visualizando participantes por sexo
spark.sql(
    '''
        SELECT
            TP_SEXO,
            COUNT(TP_SEXO) AS COUNT_SEXO
        FROM participantes_tb
        GROUP BY TP_SEXO
        ORDER BY TP_SEXO
    '''
).show(5, truncate=False)

[Stage 19:============================>                            (6 + 6) / 12]

+-------+----------+
|TP_SEXO|COUNT_SEXO|
+-------+----------+
|F      |2624621   |
|M      |1708323   |
+-------+----------+



In [35]:
#Visualizando inscritos por raça
spark.sql(
    '''
        SELECT
            TP_COR_RACA,
            COUNT(TP_COR_RACA) AS COUNT_RACA
        FROM participantes_tb
        GROUP BY TP_COR_RACA
        ORDER BY TP_COR_RACA
    '''
).show(5, truncate=False)

[Stage 22:=======================>                                 (5 + 7) / 12]

+-----------+----------+
|TP_COR_RACA|COUNT_RACA|
+-----------+----------+
|0          |50632     |
|1          |1791884   |
|2          |534653    |
|3          |1863437   |
|4          |62419     |
+-----------+----------+
only showing top 5 rows



In [36]:
# Número de Candidatos inscritos por Sexo e RACA
spark.sql(
    '''
        SELECT
            TP_SEXO,
            TP_COR_RACA,
            COUNT(1) AS QTD
        FROM participantes_tb
        GROUP BY TP_SEXO, TP_COR_RACA
        ORDER BY TP_COR_RACA
    '''
).show(truncate=False)

[Stage 25:=======================>                                 (5 + 7) / 12]

+-------+-----------+-------+
|TP_SEXO|TP_COR_RACA|QTD    |
+-------+-----------+-------+
|F      |0          |29306  |
|M      |0          |21326  |
|M      |1          |700104 |
|F      |1          |1091780|
|F      |2          |311015 |
|M      |2          |223638 |
|F      |3          |1135887|
|M      |3          |727550 |
|F      |4          |39664  |
|M      |4          |22755  |
|M      |5          |12950  |
|F      |5          |16969  |
+-------+-----------+-------+



In [37]:
## JOIN entre os dfs enem e participantes que fizeram a prova
## no municipio de Pedreiras-MA (CO_MUNICIPIO_PROVA==2108207)
## com NU_NOTA_REDACAO > 0

join_df = spark.sql(
    '''
        SELECT
            PAR.CO_MUNICIPIO_PROVA,
            PAR.TP_SEXO,
            PAR.TP_COR_RACA,
            ENE.CO_MUNICIPIO_PROVA,
            ENE.NU_NOTA_CN,
            ENE.NU_NOTA_CH,
            ENE.NU_NOTA_LC,
            ENE.NU_NOTA_MT,
            ENE.NU_NOTA_REDACAO
        FROM participantes_tb AS PAR
        INNER JOIN enem_tb AS ENE ON (PAR.CO_MUNICIPIO_PROVA = ENE.CO_MUNICIPIO_PROVA)
        WHERE
            PAR.CO_MUNICIPIO_PROVA = 2108207 AND
            ENE.CO_MUNICIPIO_PROVA = 2108207 AND
            ENE.NU_NOTA_CN IS NOT NULL AND
            ENE.NU_NOTA_CH IS NOT NULL AND
            ENE.NU_NOTA_LC IS NOT NULL AND
            ENE.NU_NOTA_MT IS NOT NULL AND
            ENE.NU_NOTA_REDACAO > 0
    '''
)

join_df.createOrReplaceTempView('participantes_enem_tb')

In [38]:
# Media notas LC e MT por sexo
spark.sql(
    '''
        SELECT
            TP_SEXO,
            MEAN(NU_NOTA_LC) AS MEDIA_NU_NOTA_LC,
            MEAN(NU_NOTA_MT) AS MEDIA_NU_NOTA_MT
        FROM participantes_enem_tb
        GROUP BY TP_SEXO
    '''
).show(truncate=False)

[Stage 29:===============================================>        (11 + 2) / 13]

+-------+------------------+------------------+
|TP_SEXO|MEDIA_NU_NOTA_LC  |MEDIA_NU_NOTA_MT  |
+-------+------------------+------------------+
|F      |504.24737931081427|502.93799999990625|
|M      |504.24737931051396|502.9379999998539 |
+-------+------------------+------------------+



In [39]:
spark.sql(
    '''
        SELECT
            TP_COR_RACA,
            MEAN(NU_NOTA_LC) AS MEDIA_NU_NOTA_LC,
            MEAN(NU_NOTA_MT) AS MEDIA_NU_NOTA_MT
        FROM participantes_enem_tb
        GROUP BY TP_COR_RACA
    '''
).show(truncate=False)

[Stage 38:===============================================>        (11 + 2) / 13]

+-----------+------------------+------------------+
|TP_COR_RACA|MEDIA_NU_NOTA_LC  |MEDIA_NU_NOTA_MT  |
+-----------+------------------+------------------+
|1          |504.2473793102544 |502.93799999988914|
|3          |504.2473793108305 |502.93799999990983|
|5          |504.247379310346  |502.9380000000007 |
|4          |504.2473793103385 |502.9379999999959 |
|2          |504.2473793102514 |502.9380000000274 |
|0          |504.24737931034093|502.9380000000018 |
+-----------+------------------+------------------+



In [40]:
spark.stop()

---

__Licensa__

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Cc_by-nc_icon.svg/1200px-Cc_by-nc_icon.svg.png" width="20%"/>



*This work is licensed under the Creative Commons Attribution-NonCommercial 4.0 International License. To view a copy of this license, visit http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.*